In [1]:
import numpy as np
import xarray as xr

xr.set_options(display_expand_data=False);

In [2]:
rng = np.random.default_rng()

In [3]:
categories = xr.DataArray(["Tech", "Finances", "Food", "Energy"], dims=["sector"])
sector_idx = rng.choice(np.arange(categories.size), size=77)
obs = xr.DataArray(
    rng.normal(size=(4, 500, 77)),
    dims=["chain", "draw", "obs_id"],
    coords={"sector_idx": (("obs_id",), sector_idx)}
)
obs

<xarray.DataArray (chain: 4, draw: 500, obs_id: 77)> Size: 1MB
0.5056 -2.795 0.9199 1.682 0.49 0.7953 ... -2.072 -0.1937 0.6796 1.309 1.851
Coordinates:
    sector_idx  (obs_id) int64 616B 2 0 0 0 2 2 0 1 3 2 ... 1 3 0 1 2 1 3 2 1 1
Dimensions without coordinates: chain, draw, obs_id

In [4]:
# all for cases below are `categories.values[index]`, what changes are the names and dimensions in output
categories.isel(sector=sector_idx)

<xarray.DataArray (sector: 77)> Size: 2kB
'Food' 'Tech' 'Tech' 'Tech' 'Food' ... 'Energy' 'Food' 'Finances' 'Finances'
Dimensions without coordinates: sector

In [5]:
categories.isel(sector=obs.coords["sector_idx"])

<xarray.DataArray (obs_id: 77)> Size: 2kB
'Food' 'Tech' 'Tech' 'Tech' 'Food' ... 'Energy' 'Food' 'Finances' 'Finances'
Coordinates:
    sector_idx  (obs_id) int64 616B 2 0 0 0 2 2 0 1 3 2 ... 1 3 0 1 2 1 3 2 1 1
Dimensions without coordinates: obs_id

In [6]:
categories.isel(sector=(("new_dim",), sector_idx))

<xarray.DataArray (new_dim: 77)> Size: 2kB
'Food' 'Tech' 'Tech' 'Tech' 'Food' ... 'Energy' 'Food' 'Finances' 'Finances'
Dimensions without coordinates: new_dim

In [7]:
categories.isel(sector=(("new_dim", "new_obs"), sector_idx.reshape(7, 11)))

<xarray.DataArray (new_dim: 7, new_obs: 11)> Size: 2kB
'Food' 'Tech' 'Tech' 'Tech' 'Food' ... 'Energy' 'Food' 'Finances' 'Finances'
Dimensions without coordinates: new_dim, new_obs

In [8]:
# not supported which is nice imo
categories.isel(sector=sector_idx.reshape(7, 11))

IndexError: Unlabeled multi-dimensional array cannot be used for indexing: sector

In [9]:
divergences_chain_idxs = xr.DataArray(rng.choice(np.arange(4), size=200), dims=["divergence_id"])
divergences_draw_idxs = xr.DataArray(rng.choice(np.arange(500), size=200), dims=["divergence_id"])

In [10]:
# obs.values[divergences_chain_idxs.values, divergences_draw_idxs.values, :]
obs.isel(chain=divergences_chain_idxs, draw=divergences_draw_idxs)

<xarray.DataArray (divergence_id: 200, obs_id: 77)> Size: 123kB
1.559 1.104 -0.1975 0.8772 -1.089 ... -1.354 -0.5761 -0.5666 -0.4062 -0.4717
Coordinates:
    sector_idx  (obs_id) int64 616B 2 0 0 0 2 2 0 1 3 2 ... 1 3 0 1 2 1 3 2 1 1
Dimensions without coordinates: divergence_id, obs_id

In [11]:
mask = obs.coords["sector_idx"] == 3
mask

<xarray.DataArray 'sector_idx' (obs_id: 77)> Size: 77B
False False False False False False False ... False False True False False False
Coordinates:
    sector_idx  (obs_id) int64 616B 2 0 0 0 2 2 0 1 3 2 ... 1 3 0 1 2 1 3 2 1 1
Dimensions without coordinates: obs_id

In [12]:
obs.isel(chain=divergences_chain_idxs, draw=divergences_draw_idxs, obs_id=mask)

<xarray.DataArray (divergence_id: 200, obs_id: 16)> Size: 26kB
-1.293 -0.715 0.3363 -0.2464 0.7372 ... 1.214 -0.1172 1.067 -0.04909 -0.5761
Coordinates:
    sector_idx  (obs_id) int64 128B 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
Dimensions without coordinates: divergence_id, obs_id

In [13]:
# I don't think it is possible to achieve this without converting the boolean mask to their integer indexes equivalent
# if it is I don't know how
mask_idx, = np.nonzero(mask.values)
obs.values[divergences_chain_idxs.values[:, None], divergences_draw_idxs.values[:, None], mask_idx[None, :]]

array([[-1.293101  , -0.71500273,  0.33632339, ...,  0.93487119,
        -0.99257948, -0.25500398],
       [-0.10294998,  0.66230001, -0.0227726 , ...,  0.03384014,
         0.67932258,  0.71239728],
       [ 0.6063118 , -1.59812246, -0.75991642, ...,  1.69820961,
         1.01326507,  0.55442668],
       ...,
       [-0.88487083, -0.556576  ,  1.482051  , ...,  0.97568127,
        -0.9281762 , -1.66976961],
       [-0.19259503,  0.23339986,  0.73059913, ...,  0.35347429,
         0.88142908, -0.7993959 ],
       [ 0.57106014,  0.05834826,  0.35048907, ...,  1.06661333,
        -0.04908627, -0.57607001]], shape=(200, 16))

In [14]:
# naive numpy breaks
obs.values[divergences_chain_idxs.values, divergences_draw_idxs.values, mask]

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (200,) (200,) (16,) 

In [15]:
# and attempted broadcasting with boolean mask instead of indexes breaks too
obs.values[divergences_chain_idxs.values[:, None], divergences_draw_idxs.values[:, None], mask[None, :]]

IndexError: too many indices

In [16]:
# not supported, also nice imo
obs.isel(chain=divergences_chain_idxs, draw=divergences_draw_idxs, obs_id=mask.rename(obs_id="other_name"))

IndexError: Boolean indexer should be unlabeled or on the same dimension to the indexed array. Indexer is on ('other_name',) but the target dimension is obs_id.

In [17]:
divergences_draw_idxs = xr.DataArray(rng.choice(np.arange(500), size=121), dims=["other_dim"])
obs.isel(chain=divergences_chain_idxs, draw=divergences_draw_idxs)

<xarray.DataArray (divergence_id: 200, other_dim: 121, obs_id: 77)> Size: 15MB
0.6385 -0.06612 -0.3086 -0.006981 0.3774 ... -0.973 -0.2469 0.4662 1.227 0.5992
Coordinates:
    sector_idx  (obs_id) int64 616B 2 0 0 0 2 2 0 1 3 2 ... 1 3 0 1 2 1 3 2 1 1
Dimensions without coordinates: divergence_id, other_dim, obs_id

In [18]:
# two consecutive calls works, but in the next example it gets quite confusing
obs.values[divergences_chain_idxs.values, :, :][:, divergences_draw_idxs.values, :].shape

(200, 121, 77)

In [19]:
# broadcasting also works and seems quite robust as long as no boolean masks/they are first converted to integer as above
obs.values[divergences_chain_idxs.values[:, None], divergences_draw_idxs.values[None, :], :].shape

(200, 121, 77)

In [20]:
# naive numpy breaks
obs.values[divergences_chain_idxs.values, divergences_draw_idxs.values, :]

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (200,) (121,) 

In [21]:
obs.isel(draw=(("new_dim",), np.arange(7)), obs_id=(("new_dim", "new_obs"), sector_idx.reshape(7, 11)))

<xarray.DataArray (chain: 4, new_dim: 7, new_obs: 11)> Size: 2kB
0.9199 0.5056 0.5056 0.5056 0.9199 ... -0.1641 -0.02136 -0.5302 -0.1641 -0.1641
Coordinates:
    sector_idx  (new_dim, new_obs) int64 616B 0 2 2 2 0 0 2 0 ... 0 0 0 0 0 0 0
Dimensions without coordinates: chain, new_dim, new_obs

In [22]:
np.allclose(
    obs.isel(draw=(("new_dim",), np.arange(7)), obs_id=(("new_dim", "new_obs"), sector_idx.reshape(7, 11))),
    obs.values[:, np.arange(7)[:, None], sector_idx.reshape(7, 11)]
)

True

In [23]:
np.allclose(
    obs.isel(draw=(("new_dim",), np.arange(7)), obs_id=(("new_dim", "new_obs"), sector_idx.reshape(7, 11))),
    obs.values[:, :, sector_idx.reshape(7, 11)][:, np.arange(7), np.arange(7), :]
)

True